# Skill estimation using graphical models

Let's see how we can use a simple model to predict a hidden "skill level" of players based on their performance in a collection of games against each other.  We need data (the games and outcomes), and a model of how skill translates into these outcomes (e.g., higher skilled players have a better chance of winning).

In [1]:
import pyGM as gm
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline         

### A simple list of games & outcomes

In [92]:
games = [
    (0,2, +1),  # P0 played P2 & won
    (0,2, +1),  # played again, same outcome
    (1,2, -1),  # P1 played P2 & lost
    (0,1, -1),  # P0 played P1 and lost
]

### Win probability and graphical model

In [93]:
nplayers = max( [max(g[0],g[1]) for g in games] )+1
nlevels = 10   # let's say 10 discrete skill levels
scale = .3     # this scales how skill difference translates to win probability

# Make variables for each player; value = skill level
X = [None]*nplayers
for i in range(nplayers):
    X[i] = gm.Var(i, nlevels)   

# Information from each game: what does Pi winning over Pj tell us?
#    Win probability  Pr[win | Xi-Xj]  depends on skill difference of players
Pwin = np.zeros( (nlevels,nlevels) )
for i in range(nlevels):
    for j in range(nlevels):
        diff = i-j                   # find the advantage of Pi over Pj, then 
        Pwin[i,j] = (1./(1+np.exp(-scale*diff)))  # Pwin = logistic of advantage

# before any games, uniform belief over skill levels for each player:
factors = [ gm.Factor([X[i]],1./nlevels) for i in range(nplayers) ]

# Now add the information from each game:
for g in games:
    P1,P2,win = g[0],g[1],g[2]
    if P1>P2: P1,P2,win=P2,P1,-win  # (need to make player IDs sorted...)
    factors.append(gm.Factor([X[P1],X[P2]], Pwin if win>0 else 1-Pwin) )

In [94]:
model = gm.GraphModel(factors)
model.makeMinimal()  # merge any duplicate factors (e.g., repeated games)

In [95]:
if model.nvar < 0:       # for very small models, we can do brute force inference:
    jt = model.joint()
    jt /= jt.sum()       # normalize the distribution and marginalize the table
    bel = [jt.marginal([i]) for i in range(nplayers)] 
else:                    # otherwise we need to use some approximate inference:
    from pyGM.messagepass import LBP, NMF
    lnZ,bel = LBP(model, maxIter=10, verbose=True)   # loopy BP
    #lnZ,bel = NMF(model, maxIter=10, verbose=True)  # Mean field

Iter 1: -2.47168477439
Iter 2: -3.00625551202
Iter 3: -3.17774815215
Iter 4: -3.20556763637
Iter 5: -3.21085081204
Iter 6: -3.21167747552
Iter 7: -3.21180102724
Iter 8: -3.21182419846
Iter 9: -3.21182781606
Iter 10: -3.21182835655


The normalization constant, $\log(Z)$, represents the (log) probability of evidence for our model, namely the probability of the observed game outcomes given our parameters, etc.  We could experiment with changing the win probability function or its scaling parameter to try to make our model better fit the data using this value.

For example, if you play with "scale" on these toy data, you'll find scale=0 (so that every game is a 50-50 chance independent of skill level) fits the data pretty well, because the few outcomes listed are not really consistent with skill determining outcome.  But, if you change the data so that there is an obvious ordering of skill (e.g., P0 then 1 and/or 2), a larger scale parameter will better fit the data.

###  Ranking players by predicted skill

In [56]:
print("Mean skill estimates: ")
print([ bel[i].table.dot(np.arange(nlevels)) for i in range(nplayers)] )

Mean skill estimates: 
[5.3063527041093286, 4.5000000014256516, 3.6936472999475161]


### Predicting match outcomes

In [63]:
i,j = 0,1
print("Estimated probability P{} beats P{} next time:".format(i,j))
# Expected value (over skill of P0, P1) of Pr[win | P0-P1]
if i<j:
    print( (bel[i]*bel[j]*gm.Factor([X[i],X[j]],Pwin)).table.sum() )
else:
    print( (bel[i]*bel[j]*gm.Factor([X[i],X[j]],1-Pwin)).table.sum() )
    
# Notes: we should probably use the joint belief over Xi and Xj, but for simplicity
#  with approximate inference we'll just use the estimated singleton marginals

Estimated probability P0 beats P1 next time:
0.547035108623
